In [2]:
# import socket and json
import socket
from time import sleep
import pickle

# Import required modules
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import time
import datetime

import re

import nltk
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/anaconda/envs/py36/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
nse_50_df = pd.read_csv('nse_50.csv')
companyName = nse_50_df['Company Name']

companyNameInlower = companyName.str.lower()
companyNameInlower.head()



0            abb india ltd.
1                  acc ltd.
2        ashok leyland ltd.
3    avenue supermarts ltd.
4        bajaj finserv ltd.
Name: Company Name, dtype: object

In [3]:
companySymbol = nse_50_df['Symbol']

companySymbolInlower = companySymbol.str.lower()
companySymbolInlower.head()

0           abb
1           acc
2      ashokley
3         dmart
4    bajajfinsv
Name: Symbol, dtype: object

In [4]:
dataForCompany =  {'col1': companyNameInlower , 'col2':companySymbolInlower}
dataFrameForCompany = pd.DataFrame(data=dataForCompany)
n = dataFrameForCompany['col1'].size
i = 0
while(i < n):
    dataFrameForCompany['col1'][i] = dataFrameForCompany['col1'][i].replace('ltd.', '')
    i += 1
    
dataFrameForCompany['col1'].head()

0            abb india 
1                  acc 
2        ashok leyland 
3    avenue supermarts 
4        bajaj finserv 
Name: col1, dtype: object

In [5]:
def CheckCompanyNameFunction(row,articletext):
    for k in row:
        if k in articletext:
            return True
        
    return False

In [6]:
def GetAllTextFromContentBody(contentBody):
    out_str = ''
    for x in contentBody:
        out_str += x.text
        
    return out_str

In [7]:
def GetDateOfArticle(publishedData):
    time_, GMT = publishedData.split('+')
    dt = datetime.datetime.strptime(time_, '%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')
    return dt

In [8]:
def GetDateTimeOfArticle(soup):
    metatags = soup.find_all('meta',attrs={'name':'publish-date'})
    for tag in metatags:
        return GetDateOfArticle(tag['content'])

In [3]:
def GetSentimentValueFromNLTK(articleText):
    
    scores = dict([('pos', 0), ('neu', 0), ('neg', 0), ('compound', 0)])
    
    if not articleText:
        return scores
    
    raw_text = articleText
    raw_text = re.sub("\n", ". ", str(raw_text))
    
    sid = SentimentIntensityAnalyzer()
    sentences = tokenize.sent_tokenize(raw_text)

    scores = dict([('pos', 0), ('neu', 0), ('neg', 0), ('compound', 0)])
    COUNT = 0
    for sentence in sentences:
        COUNT += 1
        ss = sid.polarity_scores(sentence)

        for k in sorted(ss):            
            scores[k] += ss[k]
            
    if COUNT != 0:        
        scores['pos'] /= COUNT
        scores['neu'] /= COUNT
        scores['neg'] /= COUNT
        scores['compound'] /= COUNT
        
        scores['pos'] = round(scores['pos'],2)
        scores['neu'] = round(scores['neu'],2)
        scores['neg'] = round(scores['neg'],2)
        scores['compound'] = round(scores['compound'],2)
        
    return scores

In [4]:
a = GetSentimentValueFromNLTK("My name")
print(a)

{'pos': 0.0, 'neu': 1.0, 'neg': 0.0, 'compound': 0.0}


In [10]:
def CheckCompanyNameInArticle(dataFrameForCompany,articleText):
    TragetColumn = []
    articleTextInLower = articleText.lower().split()
    for index, row in dataFrameForCompany.iterrows():
        if CheckCompanyNameFunction(row,articleTextInLower):
            TragetColumn.append(1)
        else:
            TragetColumn.append(0)
    return TragetColumn

In [ ]:
def GetDataFromLinks(column,dataFrameForCompany):
        r = requests.get(column)
        html_content = r.text
        soup = BeautifulSoup(html_content , 'lxml')
        contentbody = soup.find_all("div", id=lambda value: value and value.startswith("content-body"))
        
        articleText = GetAllTextFromContentBody(contentbody)
                
        scores = GetSentimentValueFromNLTK(articleText)
        articleDate = GetDateTimeOfArticle(soup)
                
        companyNameArray = CheckCompanyNameInArticle(dataFrameForCompany,articleText)
        DateAndScore = pd.DataFrame([articleDate,scores['pos'],scores['neu'],scores['neg'],scores['compound']])
        
        datavaluesFrame = [DateAndScore,pd.DataFrame(companyNameArray)]
        
        datavalues = pd.concat(datavaluesFrame)
        datavalues = datavalues.astype(str)
       
        print(datavalues.T.values.tolist())
        return str(datavalues.T.values.tolist())
         

In [ ]:
df = pd.read_csv('all_links_1.csv')
columnData = df['links']
i = 8528


with open('SentimentalScore_3.txt','a') as tf:
    while(i < 10535):
        x= columnData[i]
        dataFromLink = GetDataFromLinks(x,dataFrameForCompany)
        #print (dataFromLink)
        #a = pd.DataFrame(dataFromLink)
        tf.write(dataFromLink)
        tf.write('\n')
        i += 1
        
        
#with open('SentimentalScore.txt','a') as tf:
#    for x in columnData: 
#        dataFromLink = GetDataFromLinks(x,dataFrameForCompany)
#        #print (dataFromLink)
#        #a = pd.DataFrame(dataFromLink)
#       tf.write(dataFromLink)
#        tf.write('\n')
        
print("Completed")


[['2017-06-17 19:43:13', '0.15', '0.7', '0.08', '0.04', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-17 19:03:00', '0.1', '0.8', '0.02', '0.44', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-17 18:50:59', '0.11', '0.71', '0.01', '0.59', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-17 18:42:28', '0.09', '0.82', '0.04', '0.13', '0', '0', '0', '0', '0

[['2017-06-20 13:49:11', '0.03', '0.29', '0.01', '0.28', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-20 21:56:31', '0.04', '0.82', '0.05', '0.07', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-20 21:37:21', '0.02', '0.69', '0.09', '-0.44', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-20 21:30:10', '0.02', '0.76', '0.01', '0.11', '0', '0', '0', '0'

[['2017-06-21 21:56:35', '0.08', '0.82', '0.03', '0.16', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-21 21:51:18', '0.02', '0.83', '0.02', '0.1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-21 14:56:12', '0.04', '0.62', '0.01', '0.13', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-21 11:17:11', '0.03', '0.77', '0.07', '-0.25', '0', '0', '0', '0',

[['2017-06-24 20:09:52', '0.03', '0.91', '0.0', '0.15', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-24 20:04:33', '0.01', '0.74', '0.0', '0.1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-24 20:03:24', '0.11', '0.77', '0.07', '0.03', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-24 20:00:35', '0.13', '0.82', '0.01', '0.33', '0', '0', '0', '0', '0

[['2017-06-26 22:40:27', '0.08', '0.55', '0.03', '0.19', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-26 22:28:47', '0.07', '0.61', '0.06', '-0.0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-26 22:16:17', '0.08', '0.55', '0.04', '0.52', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-06-26 22:08:17', '0.06', '0.82', '0.01', '0.19', '0', '0', '0', '0',

...

[['2017-10-05 21:20:20', '0.05', '0.85', '0.01', '0.17', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-05 21:14:04', '0.14', '0.81', '0.02', '0.22', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-05 21:06:43', '0.1', '0.56', '0.01', '0.39', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-05 21:01:06', '0.07', '0.79', '0.01', '0.36', '0', '0', '0', '0', 

[['2017-10-06 12:49:02', '0.05', '0.82', '0.04', '-0.09', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-06 10:17:55', '0.15', '0.71', '0.01', '0.48', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-06 10:03:23', '0.07', '0.43', '0.0', '0.5', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-06 22:08:47', '0.1', '0.78', '0.03', '0.21', '0', '0', '0', '0', '

[['2017-10-07 22:00:54', '0.11', '0.73', '0.03', '0.43', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-07 21:22:42', '0.05', '0.63', '0.07', '0.01', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-07 21:20:40', '0.08', '0.63', '0.0', '0.33', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-07 21:13:59', '0.06', '0.79', '0.03', '0.32', '0', '0', '0', '0', 

[['2017-10-10 22:44:52', '0.02', '0.78', '0.0', '0.07', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-10 22:21:08', '0.05', '0.84', '0.05', '0.02', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-10 21:47:19', '0.02', '0.76', '0.03', '0.1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-10 21:43:08', '0.05', '0.9', '0.01', '0.13', '0', '0', '0', '0', '0

[['2017-10-10 13:29:57', '0.03', '0.62', '0.02', '0.17', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-10 12:37:03', '0.05', '0.67', '0.03', '0.12', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-10 12:33:48', '0.07', '0.72', '0.01', '0.26', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]
[['2017-10-10 12:24:00', '0.04', '0.69', '0.02', '0.15', '0', '0', '0', '0',